# IT Security - Sheet 5 "Network Security"

**Total achievable points: 20**

**Released: 12.12.2024**

**Submission Deadline: 09.01.2025**

---
Group: 128

Names and matriculation numbers of **ALL** team members: Samuel Rode (445160), Nils Maasch (445796), Pau Azpeita Bergos (443428), Gereon Geuchen (445328), Ben-Jay Huckebrink (445219)

---

**Important Information**

**Please note: This assignment is due after the Christmas holidays.**

The assignments have to be submitted by groups of 5 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`. We will then assign you to a submission group.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. In code cells, you have to remove `raise NotImplementedError()`. Please do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Please do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

Please **do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

**In this exercise an additional packages is needed.** This package is called [scapy](https://scapy.readthedocs.io/en/latest/) and is needed to investigate IPSec and TLS packets. It can be installed using `pip install scapy`. Please make sure to install the packages in the same environment as your jupyter notebook (`ipykernel`).

Bundled with this exercise, there are two different `.pcap` files. These contain a IPSec transmission and a TLS transmission respectively. Place them in the same folder as this notebook. However, you only have to submit the notebook in moodle.

*Hint: Reading documentation of modules is fun, fun, fun.*

**To access packets from a .pcap file with scapy you can use the function `rdpcap('<packet_file.pcap>')`, which returns a packet list.**

*Hint: If you want to take a look at the packets interactively, you can try out [Wireshark](https://www.wireshark.org/)!*

## Content of this Assignment

You now have a pretty good understanding of asymmetric cryptography and key establishment. We will now have a look at practical implementations and uses of these. For this purpose we will have a look at two different protocols resp. protocol families commonly used to protect data transmissions. The protocols or protocol families are named IPSec and TLS. About both you should know something from the lecture. If you can't recall these, have a look at the slides of Chapter 6 again.

## 1. IPSec Analysis (10.5 points)

For the first task of the IPSec adventures, you are already given a function that makes use of `rdpcap` to read the packets from a file specified with `path_name`. 

The function `read_pcap(path_name: str) -> list[Packet]` takes a string as the path to a `.pcap` file and returns a list of all packets that are contained in this recording of network traffic. Here, we already loaded the file `ipsec.pcap` and have a list of packets named `packets_ips`. You can use it in the subsequent tasks.

In [3]:
### You don't have to implement something here!

from scapy.all import *
load_layer("tls")

def read_pcap(path_name: str) -> list[Packet]:
    return rdpcap(path_name)

packets_ips = read_pcap("./ipsec.pcap")
packets_tls = read_pcap("./tls.pcap")

### Task 1.1 (1 point)

Write a function `test_AH(p: Packet) -> bool`, that tests, whether a packet is authenticated through the authentication header (AH) protocol.

The argument of the function is:
- `p` of type `Packet`from the `scapy` module.

The function should return a `bool` that is `True` if the packet is authenticated through an authentication header and `False` otherwise.

In [4]:
def test_AH(p: Packet) -> bool:
    # test whether a packet is using AH
    if not p.haslayer(IP):
        return False
    return p.haslayer(AH) 

In [20]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

# Output only for the first 10 packets.
for i, packet in enumerate(packets_ips):
    print(f'{i}: {test_AH(packet)}')

0: True
1: True
2: True
3: True
4: True
5: True
6: True
7: True
8: True
9: True
10: True
11: True
12: True
13: True
14: True
15: True
16: True
17: True
18: True
19: True
20: True
21: True
22: True
23: True
24: True
25: True
26: True
27: True
28: True
29: True


In [21]:
# This test just checks the output of your solution

for packet in packets_ips:
    assert type(test_AH(packet)) == bool

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.2 (1.5 points)

Now, we know which packets use the AH protocol. However, we also want to find out if it is in tunnel mode. 
Implement the function `test_AH_tunnel(p: Packet) -> bool`.
This function gets the argument:
- `p: Packet`: the packet to be investigated

The function should return a `bool` that represents wheather the packet `p` uses AH in tunnel mode. It should return `True` if it is the case and `False` otherwise.

*Hint: Have a look at the **next header** field. [There](https://en.wikipedia.org/wiki/List_of_IP_protocol_numbers) is also a handy list available. Have a look in there and look out for **IP-in-IP**.*

In [22]:
def test_AH_tunnel(p: Packet) -> bool:
    # test whether a packet is using AH tunnel using the IP-in-IP field
    if not p.haslayer(AH) or not p.haslayer(IP):
        return False
    # if proto is 4, then the outer IP packet contains another IP-Packet inside.
    return p[AH].nh == 4 
    


In [23]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

# Output only for the first 10 packets.
for i, packet in enumerate(packets_ips):
    print(f'{i}: {test_AH_tunnel(packet)}')

0: False
1: False
2: False
3: False
4: False
5: False
6: False
7: False
8: False
9: False
10: True
11: True
12: True
13: True
14: True
15: True
16: True
17: True
18: True
19: True
20: False
21: False
22: False
23: False
24: False
25: False
26: False
27: False
28: False
29: False


In [24]:
# This test just checks the output of your solution

for packet in packets_ips:
    assert type(test_AH_tunnel(packet)) == bool

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.3 (1.5 points)

Now, we also want to check if transport mode is used.
Implement the function `test_AH_transport(p: Packet) -> bool`. Again the arguments of the function are:
- `p: Packet`: the packet to be investigated

The function should return a `bool` that is `True` if AH in transport mode is used and `False` otherwise.

*Hint: Again, have a look at the **next header** field.*

In [25]:
def test_AH_transport(p: Packet) -> bool:
    if not p.haslayer(AH) or not p.haslayer(IP):
        return False
    # opposite... no IP-in-IP encapsulation and no second IP packet
    return p[AH].nh != 4 #and not p.haslayer(IP2)

In [26]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

# Output only for the first 10 packets.
for i, packet in enumerate(packets_ips):
    print(f'{i}: {test_AH_transport(packet)}')

0: True
1: True
2: True
3: True
4: True
5: True
6: True
7: True
8: True
9: True
10: False
11: False
12: False
13: False
14: False
15: False
16: False
17: False
18: False
19: False
20: True
21: True
22: True
23: True
24: True
25: True
26: True
27: True
28: True
29: True


In [27]:
# This test just checks the output of your solution

for packet in packets_ips:
    assert type(test_AH_transport(packet)) == bool

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.4 (2 points)

Apart from the above tasks where IPSec is used to protect the integrity of the packets, there is also the option to use encryption using the Encapsulating Security Payload (ESP) protocol. In the following we want to determine of ESP was used or not.

Write a function `test_ESP(p: Packet) -> bool`. Again, the arguments to this function are:
- `p: Packet`: the packet to be investigated

The function should return a `bool` that is `True` if the packet `p` is protected with the encapsulation security protocol (ESP) and `False` otherwise. Note, that the packet may also use the AH protocol.

In [28]:
def test_ESP(packet) -> bool:
   if not packet.haslayer(IP):
       return False

   # Yes, ESP is in the innermost `Raw` packet and not an extra
   # layer by itself, so `ESP in packet` does _not work_
   # (even though it really should...)
   return packet[IP].proto == 50 or packet[AH].nh == 50

In [29]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

# Output only for the first 10 packets.
for i, packet in enumerate(packets_ips):
    print(f'{i}: {test_ESP(packet)}')

0: False
1: False
2: False
3: False
4: False
5: False
6: False
7: False
8: False
9: False
10: False
11: False
12: False
13: False
14: False
15: False
16: False
17: False
18: False
19: False
20: True
21: True
22: True
23: True
24: True
25: True
26: True
27: True
28: True
29: True


In [30]:
# This test just checks the output of your solution

for packet in packets_ips:
    assert type(test_ESP(packet)) == bool

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.5 (2 point)

Can you detect, whether ESP is used in tunnel or transport mode? Explain your answer for both cases.

In tunnelmode, the entire IP-Packet gets encrypted and encapsulated in a new IP-Packet. Also the nextHeader field is, unlike for AH, considered confidential. So the nextHeader field is in the ESP-Trailer, which gets encrypted and is not visible from the outside. Since the payload is encrypted, there's no way to determine if it contains another IP-Packet or not. So you can't tell for tunnelmode.
Its similar for transport mode. The payload is encrypted, so you can't tell if it contains another IP-Packet or not.

### Task 1.6 (1 point)

ESP does provide authentication. Why does AH even exist?

There might be some use cases where you don't want to encrypt the payload, but still want to authenticate it. E.g. if you want to check incoming packets for potentially malicious content, but don't want to decrypt the payload. This saves the _overhead_ associated with en-/decryption, making AH "more lightweight" than ESP.

### Task 1.7 (1.5 points)

From the lecture, you know that AH and ESP can be used together. Your task is now to create a `scapy`packet that makes use of this construction.

Implement the function `create_AH_ESP_tunnel() -> Packet`. This function takes no arguments and returns a IPv4 packet with all the correct headers in the correct order that are needed to use ESP and AH in conjunction. Do not forget the actual payload. In this task, the actual packet should not contain any data, just create the header structure. So, you don't have to provide any values here. You can use `scapy` to construct such a packet in the following way: `packet = IP()/TCP()`. If you arfe unsure what to include here, have a look in the `.pcap` file using wireshark.

In [31]:
def create_AH_ESP_tunnel() -> Packet:
    # create a packet with AH and ESP and IP-in-IP encapsulation
    ip_header = IP()
    ah_header = AH()
    esp_header = ESP()
    payload = TCP()/Raw(b'') # empty payload
    return ip_header/ah_header/esp_header/payload

In [32]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(create_AH_ESP_tunnel())
create_AH_ESP_tunnel()

IP / AH / ESP / TCP ftp_data > www_http S / Raw


<IP  proto=ah |<AH  |<ESP  |<TCP  |<Raw  |>>>>>

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 2. TLS Analysis (4.5 points)

Now, we looked at some IPSec packets, but you also learned about TLS. Hence, we want to take a look at some TLS packets too.
You will now need the other pcap file named `tls.pcap` from Moodle. You can still reuse the `read_pcap` function from above though! We already read in the file, which is available as a packet list in `packets_tls`.

### Task 2.1 (1.5 point)
We are now especially interested in the TLS handshake. Hence, your first task is to write a function which determines for a given packet whether it belongs to a TLS handshake or not.

Implement the function `check_TLS_handshake(p: Packet) -> bool` with the arguments:
- `p: Packet`: the packet to be investigated

The function should return `True` if the packet belongs to a TLS handshake and `False` otherwise. 

*Hint: The TLS record type might be of interest. Have a look [here](https://en.wikipedia.org/wiki/Transport_Layer_Security#TLS_record).*

In [33]:
def check_TLS_handshake(p: Packet) -> bool:
    # check if the packet is a TLS handshake packet
    if not p.haslayer(TLS):
        return False
    return p[TLS].type == 22 # 22 is the type for handshake according to the link.

In [34]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

# Output only for the first 10 packets.
for i, packet in enumerate(packets_tls[:10]):
    print(f'{i}: {check_TLS_handshake(packet)}')

0: False
1: False
2: False
3: True
4: False
5: True
6: False
7: True
8: False
9: False


In [35]:
# This test just checks the output of your solution

for packet in packets_tls:
    assert type(check_TLS_handshake(packet)) == bool

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.2 (3 points)

You learned that both the client and the server exchange random values during the handshake. We want to know how many bytes these take.

Implement the function `get_random_length(path: str) -> int` with the arguments:
- `path: str`: path to the `.pcap`file of interest (in our case `tls.pcap`)

The function should return an `int` that represents the total number of all random **bytes** that are exchanged in the TLS exchange given in the file at the path.

*Hint: The random value each party sends consists of a timestamp and some random bytes. We want to know the number of bytes including the timestamp since this is the random that is sent to the other party!*

*Hint: You can also verify this random by looking at the handshake with Wireshark.*

In [36]:
from datetime import datetime
from math import ceil

def get_random_length(path: str) -> int:
    # total number of all random bytes exchanged in the file including the timestamp
    packets = read_pcap(path)
    random_bytes = 0
    for packet in packets:
        # Only `ClientHello` and `ServerHello` contain the asked for
        # random bytes in the specified formats
        # As per RFCs 5246 (TLS 1.2) and 8446 (TLS 1.3), these fields are
        # exactly 32 bytes long
        #
        # Thus, per Hello message, we just count up 32

        if packet.haslayer(TLS):
            # Check for ClientHello
            if packet.haslayer(TLSClientHello):
                random_bytes += 32
            # Check for ServerHello    
            elif packet.haslayer(TLSServerHello):
                random_bytes += 32
    
    return random_bytes



In [37]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(get_random_length("tls.pcap"))

64


In [38]:
# This test just checks the output of your solution

assert type(get_random_length("tls.pcap")) == int

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 3. IPSec vs. TLS (4 points)

We learned about two network security protocols. But why are there two? What are the differences? With the following tasks, you should convince me that there is a need for both of them! 

### Task 3.1 (2 point)

Compare the typical use cases of both TLS and IPSec.

First of all, IPSec operates on IP-Packet level, so it really protects IP-Packets. TLS on the other hand operates over the transport layer so it protects TCP Segments. A typical use case of IP-Sec is to secure the communication between two networks or routers; it can also be used for VPNs. This means that IPSec is typically used where securing **all traffic sent** between two computers - _regardless of application_ - is desired.


TLS is for transport layer protection of a specific application, e.g. a browser and a webserver. In the lecture, we also saw that it can be used for mail. This means that TLS is typically used where one wants to choose security on a **per-application basis**.

### Task 3.2 (2 points)

For both IPSec and TLS, name advantages and disadvantages compared to each other.

TLS is ideal for securing individual sessions, e.g. between a browser and a webserver. IPSec is better suited for securing entire network segments. Its ideal for connecting branches or securing all traffic between networks. 
Since IPSec operates on the network layer, IPSec provides its services to all protocols and applications above. This means that applications dont need to be modified to use IPSec. With Traffic Selectors, you can also specify which traffic should be protected.
TLS operates on the transport layer, so it only protects the application that use it explicitly. 
IPSec might also be more efficient, since it operates on a lower level. TLS might have more overhead because of its session establishment etc.. 


## 4. Exam Example Tasks (0.5 points)

> Note: This task is awarded with 0.5 points only if it has been seriously addressed, regardless of whether the answer is correct.

### Task 4.1 - IPSec

Assume the following IP packet given:

```
+--------------------+---------+
| Original IP Header | Payload |
+--------------------+---------+
```

*Hint: You can copy the notation above and adapt it in the tasks.*

#### a)

**Draw/List** all contents of the packet (including all headers and trailers) if it would be sent with AH in tunnel mode. Additionally, **mark** which parts are encrypted and which parts are integrity protected.

I = Integrity Protected 
E = Encrypted
```
+--------------------+---------------------+--------------------+---------+
| New IP Header (I)  | AH Header (I)       | Original IP Header(I)| Payload (I) |
+--------------------+---------------------+--------------------+---------+
```

#### b)

Now, the packet is protected with ESP in transport mode. **Mark/List** which parts are encrypted and which ones are integrity protected.

```
+--------------------+-----+---------+-------------+-----+
| Original IP Header | ESP | Payload | ESP Trailer | MAC |
+--------------------+-----+---------+-------------+-----+
```

I = Integrity Protected 
E = Encrypted
```
+--------------------+-----  +---------     +-------------     +-----+
| Original IP Header | ESP(I)| Payload(I,E) | ESP Trailer(I,E) | MAC |
+--------------------+-----  +---------     +-------------     +-----+
```

### Task 4.2 - TLS

In the figure below, a TLS 1.3 handshake in a simplified form is illustrated.

![](./tls.png)

**Answer** the following questions. **Justify** your answer.

#### a)

Does the depicted TLS 1.3 handshake use key transport or key agreement?

Key Agreement, since the client and the server both send their DH-Values to each other, which are then used to calculate the shared secret.

#### b)

From which perspective is key freshness guaranteed?

From both perspectives. When one of them knows they chose their DH-Value random, they can be sure that the key is fresh since its computed on both values. So even if you might think the other party didnt choose a random value, you can be sure that the key is fresh.

#### c)

What has to be added in which message(s) so that the server is authenticated through a certificate.

In the server's first message, it should sign the DH-Value it sends out with its private key. But it should also sign the DH-Value it already received because this value acts as a challenge and the server is responding that way. This way, the user knows that its not just someone replaying the servers message, but the actual server. In addition to that, it should also send its certificate.
Assuming the client has the pk of the certificate authority, it can verify the certificate of the server and verify the signature it received.

#### d)

What happens if an attacker would change the SupportedAlgos parameter in the first message?

It would be noticed since the 'finished' part of the servers message includes the MAC on everything that was sent before. So the client would notice that the server received something different than what it sent.

## 5. Feedback (0.5 points)

> Note: This task is awarded with 0.5 points only if there is feedback in any form to this exercise. It is also okay to state what was especially nice or rewarding. Literally just writing "anything" is not enough. Any feedback will improve the assignments.

Nicely done, another successful assignment by you! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write some feedback. But don't worry, we do not grade the content itself!

I liked the coding part of the last exercises, but for this one, I didnt like that it depended on the pcap files and the library. I dont really see the point in learning specific libraries for these kind of tasks. IMO, it would be better to have more theoretical tasks, which dont depend on specific libraries.